In [87]:
from keras.models import Model
from keras.layers import Input,LSTM,Dense
import numpy as np

In [157]:
batch_size=64

In [158]:
latent_dim=256

In [159]:
num_samples=200000

In [91]:
data_path='/home/eeswar/Downloads/datasets/fra.txt'

In [92]:
input_texts=[]
target_texts=[]

In [93]:
input_characters=set()
target_characters=set()

In [61]:
with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')

In [94]:
for i in lines[:min(num_samples,len(lines)-1)]:
    input_text,target_text,_=i.split('\t')
    target_text='\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
            input_characters.add(char)
    for char in target_text:
            target_characters.add(char)

In [95]:
len(target_characters)

91

In [96]:
len(input_characters)

70

In [97]:
len(input_texts)

10000

In [98]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

In [99]:
num_encoder_tokens=len(input_characters)

In [100]:
num_decoder_tokens=len(target_characters)

In [101]:
max_encoder_seq_length=max(len(i) for i in input_texts)
max_decoder_seq_length=max(len(i) for i in target_texts)

In [102]:
input_token_index=dict(
    [char,i] for i,char in enumerate(input_characters)
)

In [103]:
target_token_index=dict(
    [char,i] for i,char in enumerate(target_characters)
)

In [104]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [105]:
encoder_input_data=np.zeros(
    (len(input_texts),max_encoder_seq_length,num_encoder_tokens),
    dtype='float32'
)

In [106]:
decoder_input_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype='float32'
)

In [107]:
decoder_target_data=np.zeros(
    (len(input_texts),max_decoder_seq_length,num_decoder_tokens),
    dtype='float32'
)

In [110]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token_index[char]]=1
    encoder_input_data[i,t+1:,input_token_index[' ']]=1
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token_index[char]]=1
        if t>0:
            decoder_target_data[i,t-1,target_token_index[char]]=1
    decoder_input_data[i,t+1:,target_token_index[' ']]=1
    decoder_target_data[i,t:,target_token_index[' ']]=1

In [112]:
encoder_input_data[0].shape

(14, 70)

In [113]:
encoder_inputs=Input(shape=(None,num_encoder_tokens))

In [115]:
 encoder=LSTM(latent_dim,return_state=True)

In [116]:
enoder_outputs,state_h,state_c=encoder(encoder_inputs)

In [117]:
encoder_states=[state_h,state_c]

In [118]:
decoder_inputs=Input(shape=(None,num_decoder_tokens))

In [120]:
decoder=LSTM(latent_dim,return_sequences=True,return_state=True)

In [121]:
decoder_outputs,_,_=decoder(decoder_inputs,initial_state=encoder_states)

In [122]:
decoder_den=Dense(num_decoder_tokens,activation='softmax')

In [123]:
decoder_outputs=decoder_den(decoder_outputs)

In [124]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [125]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [127]:
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,
         batch_size=64,
         epochs=100,
         validation_split=0.2)

2025-05-21 11:27:06.305952: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.
2025-05-21 11:27:06.587671: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 171808000 exceeds 10% of free system memory.


Epoch 1/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 49s 390ms/step - accuracy: 0.7479 - loss: 0.9326 - val_accuracy: 0.7305 - val_loss: 0.9551
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 46s 365ms/step - accuracy: 0.7682 - loss: 0.8405 - val_accuracy: 0.7583 - val_loss: 0.8599
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 357ms/step - accuracy: 0.7893 - loss: 0.7597 - val_accuracy: 0.7689 - val_loss: 0.7896
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 42s 339ms/step - accuracy: 0.8049 - loss: 0.6795 - val_accuracy: 0.7984 - val_loss: 0.7142
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 322ms/step - accuracy: 0.8156 - loss: 0.6362 - val_accuracy: 0.8024 - val_loss: 0.6849
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 317ms/step - accuracy: 0.8237 - loss: 0.6035 - val_accuracy: 0.8100 - val_loss: 0.6526
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 322ms/step - accuracy: 0.8305 - loss: 0.5786 - val_accuracy: 0.8165 - val_loss: 0.6307
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 321ms/step - accuracy: 0.8347 -

In [128]:
encoder_model=Model(encoder_inputs,encoder_states)

In [130]:
decoder_state_input_h=Input(shape=(latent_dim,))

In [131]:
latent_dim

256

In [132]:
decoder_state_input_c=Input(shape=(latent_dim,))

In [134]:
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]

In [136]:
decoder_outputs,state_h,state_c=decoder(
    decoder_inputs,initial_state=decoder_states_inputs
)

In [137]:
decoder_states=[state_h,state_c]

In [138]:
decoder_outputs=decoder_den(decoder_outputs)

In [139]:
decoder_model=Model(
    [decoder_inputs]+decoder_states_inputs,
    [decoder_outputs]+decoder_states
)

In [140]:
reverse_input_char_index=dict(
    (i,char) for char,i in input_token_index.items()
)

In [141]:
reverse_target_char_index=dict(
    (i,char) for char,i in target_token_index.items()
)

In [146]:
def decode_sequence(input_seq):
    state_value=encoder_model.predict(input_seq)
    target_seq=np.zeros((1,1,num_encoder_tokens))
    target_seq[0,0,target_token_index['\t']]=1
    stop_condition=False
    decoded_sentence=''
    while not stop_condition:
        output_tokens,h,c=decoder_model.predict(
            [target_seq]+states_value
        )
        sampled_token_index=np.argmax(output_tokens[0,-1:])
        sampled_char=reverse_target_char_index[sampled_token_index]
        decoded_sentence+=sampled_char
        if(sampled_char=='\n' or len(decoded_sentence)>max_decoder_seq_length):
            stop_condition=True
        target_seq=np.zeros((1,1,num_decoder_tokens))
        target_seq[0,0,sampled_token_index]=1
        states_value=[h,c]
    return deoded_sentence

In [151]:
for seq_index in range(100):
    input_seq=encoder_input_data[seq_index:seq_index+1]
    decoded_sentence=decode_sequence(input_seq)
    print('-')
    print('Input sentence:',input_texts[seq_index])
    print('Decoded sentence:',decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
-
Input sentence: Go.
Decoded sentence: Boursuis.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
-
Input sentence: Go.
Decoded sentence: Boursuis.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [150]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1. 
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value
        )
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]
    return decoded_sentence

In [152]:
new_word = "hello"

# Assume `input_token_index` is a dictionary mapping characters to integers.
encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for t, char in enumerate(new_word):
    if char in input_token_index:
        encoder_input[0, t, input_token_index[char]] = 1.
    else:
        print(f"Unknown character: {char}")


In [153]:
decoded_sentence = decode_sequence(encoder_input)
print("Decoded:", decoded_sentence)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Decoded: Nendrez les feutes.



In [154]:
test_word = "hello"

encoder_input = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
for t, char in enumerate(test_word):
    if char in input_token_index:
        encoder_input[0, t, input_token_index[char]] = 1.
    else:
        print(f"Character '{char}' not in vocabulary!")


In [155]:
decoded_sentence = decode_sequence(encoder_input)
print("Input:", test_word)
print("Predicted French:", decoded_sentence)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Input: hello
Predicted French: Nendrez les feutes.

